In [ ]:
!pip install streamlit altair pandas numpy pyngrok
!pip install openai==0.28

In [ ]:
%%writefile dashboard_dt.py

import os
import openai
import streamlit as st
import pandas as pd
import numpy as np
import altair as alt
import pydeck as pdk
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression

# --- CONFIGURATION DE LA PAGE ---
openai.api_key = "openaikey"
st.set_page_config(page_title="Jumeau Numérique Qualité de l'Air", layout="wide")

# --- CSS PERSONNALISÉ POUR LE DESIGN ---
st.markdown("""
<style>
  section[data-testid="stSidebar"] > div:first-child {
    padding-top: 2rem;
    width: 260px;
    background-color: #1f2937;
    color: white;
  }
  .stSidebar .element-container .stMetric {
    background-color: #111827;
    color: white;
    margin-bottom: 12px;
  }
  .stButton > button {
    background-color: #3b82f6;
    color: white;
    border-radius: 6px;
    padding: 0.4rem 1rem;
    font-size: 15px;
    margin-top: 1rem;
  }
  .block-container {
    padding-top: 0;
  }
  .stMetric {
    background-color: #f3f4f6;
    padding: 0.6rem;
    border-radius: 10px;
    margin-bottom: 10px;
    font-size: 14px;
  }
  h1, h2, h3, h4 {
    margin-top: 1.5rem;
    margin-bottom: 0.7rem;
  }
  .legend {
    display: flex;
    gap: 1rem;
    margin-bottom: 1rem;
  }
  .legend-item {
    display: flex;
    align-items: center;
    gap: 0.3rem;
  }
  .legend-color {
    width: 16px;
    height: 16px;
    border-radius: 4px;
  }
</style>
""", unsafe_allow_html=True)

# --- AJOUT DES INDICATEURS CLÉS DANS LA BARRE LATÉRALE ---
with st.sidebar:
    st.markdown("### 📌 Performance")
    # Sparkline : progression de la précision
    dates_dt = pd.date_range(end=datetime.today(), periods=7).to_pydatetime().tolist()
    scores_dt = np.round(np.linspace(75, 90, len(dates_dt)) + np.random.randn(len(dates_dt)) * 1.5, 1)
    df_dt = pd.DataFrame({"Date": dates_dt, "Précision DT (%)": scores_dt})
    st.sidebar.metric("Précision DT (Dernier)", f"{scores_dt[-1]} %", f"{(scores_dt[-1] - scores_dt[-2]):+.1f} %")
    st.sidebar.line_chart(df_dt.set_index("Date"), height=100)
    # Expander pliable par défaut pour les KPIs
    with st.expander("Voir les KPIs", expanded=False):
        # Génération de données synthétiques pour chaque KPI
        dates_dt = pd.date_range(end=datetime.today(), periods=7).to_pydatetime().tolist()
        # KPI 1 : Précision DT (%)
        scores_dt = np.round(np.linspace(75, 90, len(dates_dt)) + np.random.randn(len(dates_dt)) * 1.5, 1)
        df_prec = pd.DataFrame({"Date": dates_dt, "Précision (%)": scores_dt})
        st.markdown("**Précision DT** : performance moyenne du modèle (%)")
        chart_prec = (
            alt.Chart(df_prec)
            .mark_line(point=True, strokeWidth=2)
            .encode(
                x=alt.X("Date:T", title=""),
                y=alt.Y("Précision (%):Q", title=""),
                tooltip=["Date:T", "Précision (%):Q"]
            )
            .properties(width=200, height=100)
        )
        st.altair_chart(chart_prec, use_container_width=True)

        # KPI 2 : RMSE (Erreur quadratique moyenne)
        mois_kpi = ["Mars", "Avr", "Mai", "Juin", "Juil", "Août", "Sept"]
        rmse_vals = [4.5, 4.2, 4.0, 3.9, 3.7, 3.6, 3.5]
        df_rmse = pd.DataFrame({"Mois": mois_kpi, "RMSE": rmse_vals})
        st.markdown("**RMSE** : écart quadratique moyen (µg/m³)")
        chart_rmse = (
            alt.Chart(df_rmse)
            .mark_line(point=True, strokeWidth=2)
            .encode(
                x=alt.X("Mois:N", title=""),
                y=alt.Y("RMSE:Q", title=""),
                tooltip=["Mois:N", "RMSE:Q"]
            )
            .properties(width=200, height=100)
        )
        st.altair_chart(chart_rmse, use_container_width=True)

        # KPI 3 : MAE (Erreur absolue moyenne)
        mae_vals = [2.7, 2.5, 2.3, 2.2, 2.0, 1.9, 1.8]
        df_mae = pd.DataFrame({"Mois": mois_kpi, "MAE": mae_vals})
        st.markdown("**MAE** : erreur absolue moyenne (µg/m³)")
        chart_mae = (
            alt.Chart(df_mae)
            .mark_line(point=True, strokeWidth=2, color="#FFA500")
            .encode(
                x=alt.X("Mois:N", title=""),
                y=alt.Y("MAE:Q", title=""),
                tooltip=["Mois:N", "MAE:Q"]
            )
            .properties(width=200, height=100)
        )
        st.altair_chart(chart_mae, use_container_width=True)

        # KPI 4 : Uptake (%)
        uptake_vals = [70, 75, 80, 82, 85, 87, 90]
        df_uptake = pd.DataFrame({"Mois": mois_kpi, "Uptake (%)": uptake_vals})
        st.markdown("**Uptake** : pourcentage d’utilisateurs actifs (%)")
        chart_uptake = (
            alt.Chart(df_uptake)
            .mark_line(point=True, strokeWidth=2, color="#20c997")
            .encode(
                x=alt.X("Mois:N", title=""),
                y=alt.Y("Uptake (%):Q", title=""),
                tooltip=["Mois:N", "Uptake (%):Q"]
            )
            .properties(width=200, height=100)
        )
        st.altair_chart(chart_uptake, use_container_width=True)

        # KPI 5 : Efficience (%)
        efficience_vals = [75, 77, 80, 82, 84, 86, 88]
        df_efficience = pd.DataFrame({"Mois": mois_kpi, "Efficience (%)": efficience_vals})
        st.markdown("**Efficience** : efficacité globale (temps, coûts) (%)")
        chart_efficience = (
            alt.Chart(df_efficience)
            .mark_line(point=True, strokeWidth=2, color="#6f42c1")
            .encode(
                x=alt.X("Mois:N", title=""),
                y=alt.Y("Efficience (%):Q", title=""),
                tooltip=["Mois:N", "Efficience (%):Q"]
            )
            .properties(width=200, height=100)
        )
        st.altair_chart(chart_efficience, use_container_width=True)

# --- CONSTANTES ---
REGIONS = {
    "Centre-Val de Loire": (47.5, 1.5),
    "Hauts-de-France": (50.4, 3.0),
    "Auvergne-Rhône-Alpes": (45.75, 4.85),
    "Normandie": (49.1, -0.4)
}
POLLUANTS = ["O3", "PM10", "NH3", "PM2.5", "CH4"]
PROFILS = ["Public", "Expert"]
INTENTS = ["Décrire", "Comparer", "Prévoir", "Recommander"]
FORECASTS = ["Court Terme", "Moyen Terme", "Long Terme"]
SOURCES = ["Lig'Air", "Satellite", "Météo-Sol", "ECMWF", "Flexpart"]

STRUCTURES = {
    "Pyramide Inversée": {"sections": ["Point Clé", "Détails Complémentaires", "Contexte"]},
    "Trois Actes": {"sections": ["Contexte", "Observations", "Recommandations"]},
    "Empilement de Blocs": {"sections": ["Introduction", "Visualisation Principale", "Tendances", "Conclusion"]},
    "Château d'Eau": {"sections": ["Introduction", "Visuel Historique", "Interprétation", "Recommandations", "Conclusion"]},
}
MAPPING_STRUCTURE = {
    "Décrire": "Empilement de Blocs",
    "Comparer": "Pyramide Inversée",
    "Prévoir": "Château d'Eau",
    "Recommander": "Trois Actes"
}

VIZ_RECOMMANDEES = {
    "Décrire": {"Public": ["Courbe Multi-lignes"], "Expert": ["Courbe Multi-lignes", "Tableau"]},
    "Comparer": {"Public": ["Courbe Multi-lignes"], "Expert": ["Courbe Multi-lignes", "Barres Moyennes"]},
    "Prévoir": {"Public": ["Courbe Historique + Prévisions"], "Expert": ["Courbe Historique + Prévisions", "Tableau"]},
    "Recommander": {"Public": ["Tableau Actions"], "Expert": ["Tableau Actions", "Barres Colorées"]}
}


# --- FONCTIONS UTILES ---
@st.cache_data
def charger_donnees(polls, debut, fin):
    """
    Génère un DataFrame simulé pour chaque polluant entre les dates données.
    """
    dates = pd.date_range(debut, fin, freq="D")
    df = pd.DataFrame()
    for p in polls:
        tmp = pd.DataFrame({
            "date": dates,
            "valeur": np.random.uniform(20, 80, len(dates))
        })
        tmp["polluant"] = p
        df = pd.concat([df, tmp], ignore_index=True)
    return df.reset_index(drop=True)


def generer_visu(df, intention, audience, polluants_sel, forecast_sel):
    """
    Retourne une visualisation Altair selon l'intention et le profil utilisateur.
    Pour 'Décrire' ou 'Comparer', on trace une courbe multi-lignes.
    Pour 'Prévoir', on trace une courbe historique vs prévision à deux couleurs.
    """
    if df is None or df.empty:
        return None

    # --- Cas "Décrire" ou "Comparer" multi-lignes ---
    if intention in ["Décrire", "Comparer"] and polluants_sel:
        df_f = df[df["polluant"].isin(polluants_sel)].copy()
        if df_f.empty:
            return None

        chart = (
            alt.Chart(df_f)
            .mark_line(strokeWidth=3)
            .encode(
                x=alt.X("date:T", title="Date"),
                y=alt.Y("valeur:Q", title="Concentration (µg/m³)"),
                color=alt.Color("polluant:N", title="Polluant"),
                tooltip=["date:T", "polluant:N", "valeur:Q"]
            )
            .properties(width="container", height=350)
            .interactive()
        )
        return chart

    # --- Cas "Prévoir" : courbe historique + prévision ---
    if intention == "Prévoir" and polluants_sel:
        p = polluants_sel[0]
        base = df[df["polluant"] == p].sort_values("date").copy()
        if base.shape[0] < 2:
            return None

        base["indice"] = np.arange(len(base))
        modele = LinearRegression().fit(base[["indice"]], base[["valeur"]])
        jours_futurs = {"Court Terme": 2, "Moyen Terme": 7, "Long Terme": 15}[forecast_sel]
        fut_idx = np.arange(len(base), len(base) + jours_futurs)
        preds = modele.predict(fut_idx.reshape(-1, 1)).flatten()
        fut_dates = pd.date_range(base["date"].max() + timedelta(days=1), periods=jours_futurs)

        hist = base[["date", "valeur"]].assign(type="Historique", polluant=p)
        fut = pd.DataFrame({
            "date": fut_dates,
            "valeur": preds,
            "type": "Prévision",
            "polluant": p
        })
        total = pd.concat([hist, fut], ignore_index=True)

        chart = (
            alt.Chart(total)
            .mark_line(strokeWidth=3)
            .encode(
                x=alt.X("date:T", title="Date"),
                y=alt.Y("valeur:Q", title=f"{p} (µg/m³)"),
                color=alt.Color(
                    "type:N",
                    title="Type",
                    scale=alt.Scale(domain=["Historique", "Prévision"], range=["#0d6efd", "#dc3545"])
                ),
                tooltip=["date:T", "valeur:Q", "type:N"]
            )
            .properties(width="container", height=350)
        )
        return chart

    return None


def saison_from_date(d):
    """
    Renvoie une chaîne de caractères pour la saison (Hiver, Printemps, Été, Automne)
    selon le mois de la date fournie.
    """
    m = d.month
    if m in (12, 1, 2):
        return "hiver"
    if m in (3, 4, 5):
        return "printemps"
    if m in (6, 7, 8):
        return "été"
    return "automne"


def generer_story_vraie(df, intention, structure, audience, region, debut, fin, polluants_sel, forecast_sel):
    """
    Construit un dictionnaire représentant chaque section de la 'Data Story'.
    Chaque titre est attractif, chaque texte est narratif, et on fournit
    une visualisation ou un tableau selon le contexte.
    """
    story = {}
    sections = STRUCTURES[structure]["sections"]
    saison = saison_from_date(debut)

    # --- Bloc contextuel général (paragraphe) ---
    contexte_regional = (
        f"Contexte régional ({region}) – Saison : {saison}. "
        "Pendant cette période, le trafic routier, l’industrie locale et, en hiver, le chauffage résidentiel "
        "génèrent la majeure partie des émissions. Les vents et les inversions de températures influencent la dispersion des polluants.\n\n"
    )

    # --- Cas "Décrire" multi-polluants ---
    if intention == "Décrire":
        if not polluants_sel:
            story[sections[0]] = {
                "titre": "Aucune sélection de polluant",
                "texte": "Veuillez sélectionner au moins un polluant pour générer la data story.",
                "chart": None
            }
            return story

        # Calcul des statistiques pour chaque polluant
        stats = []
        for p in polluants_sel:
            df_p = df[df["polluant"] == p]
            if df_p.empty:
                stats.append({
                    "polluant": p,
                    "moyenne": None,
                    "minimum": None,
                    "maximum": None,
                    "variation": None
                })
            else:
                moyenne = round(df_p["valeur"].mean(), 1)
                minimum = round(df_p["valeur"].min(), 1)
                maximum = round(df_p["valeur"].max(), 1)
                variation = None
                if len(df_p) >= 2:
                    debut_val = df_p.iloc[0]["valeur"]
                    fin_val = df_p.iloc[-1]["valeur"]
                    variation = round((fin_val - debut_val) / debut_val * 100, 1) if debut_val != 0 else 0
                stats.append({
                    "polluant": p,
                    "moyenne": moyenne,
                    "minimum": minimum,
                    "maximum": maximum,
                    "variation": variation
                })

        # Section 1 : Tableau Statistiques
        df_stats = pd.DataFrame(stats)
        df_stats_aff = df_stats.copy()
        for col in ["moyenne", "minimum", "maximum", "variation"]:
            df_stats_aff[col] = df_stats_aff[col].apply(
                lambda x: f"{x} µg/m³" if isinstance(x, (int, float)) and col != "variation"
                          else (f"{x} %" if isinstance(x, (int, float)) and col == "variation" else "N/A")
            )
        texte_intro = (
            contexte_regional +
            f"Du {debut.strftime('%d/%m/%Y')} au {fin.strftime('%d/%m/%Y')}, les indicateurs pour chaque polluant sont :"
        )
        story[sections[0]] = {
            "titre": f"Statistiques pour {', '.join(polluants_sel)}",
            "texte": texte_intro,
            "chart": df_stats_aff
        }

        # Section 2 : Courbe Multi-lignes
        visu = generer_visu(df, "Décrire", audience, polluants_sel, None)
        texte_visu = (
            "Évolution journalière des concentrations pour chaque polluant. Les lignes colorées permettent d’identifier "
            "les pics et les creux au fil du temps."
        )
        story[sections[1]] = {
            "titre": f"Tendances Journalières ({', '.join(polluants_sel)})",
            "texte": texte_visu,
            "chart": visu
        }

        # Section 3 : Analyse des Tendances
        lignes_tend = []
        for s in stats:
            p = s["polluant"]
            var = s["variation"]
            if var is None:
                lignes_tend.append(f"- {p} : données insuffisantes pour la tendance.")
            else:
                sens = "hausse" if var > 0 else "baisse" if var < 0 else "stabilité"
                lignes_tend.append(f"- {p} : {sens} de {abs(var)} % sur la période.")
        texte_tend = "Analyse des tendances pour chaque polluant :\n" + "\n".join(lignes_tend)
        story[sections[2]] = {
            "titre": "Analyse des Tendances",
            "texte": texte_tend,
            "chart": None
        }

        # Section 4 : Recommandations
        recos = []
        for s in stats:
            p = s["polluant"]
            moy = s["moyenne"]
            if moy is None:
                recos.append(f"- {p} : aucune donnée disponible.")
            else:
                if moy >= 60:
                    recos.append(f"- {p} ({moy} µg/m³) dépasse le seuil critique → limiter les activités extérieures.")
                else:
                    recos.append(f"- {p} ({moy} µg/m³) est sous le seuil critique → maintenir une bonne surveillance.")
        texte_reco = "Recommandations générales :\n" + "\n".join(recos)
        story[sections[3]] = {
            "titre": "Recommandations",
            "texte": texte_reco,
            "chart": None
        }

        return story

    # --- Cas "Comparer" ---
    if intention == "Comparer":
        if df.empty or len(polluants_sel) < 2:
            story[sections[0]] = {
                "titre": "Comparatif impossible",
                "texte": "Pour comparer, sélectionnez au moins deux polluants.",
                "chart": None
            }
            return story

        # Calcul des moyennes
        moyennes = {
            poll: round(val, 1)
            for poll, val in df[df["polluant"].isin(polluants_sel)].groupby("polluant")["valeur"].mean().items()
        }
        meilleur = max(moyennes, key=moyennes.get)
        plus_bas = min(moyennes, key=moyennes.get)

        # Section 1 : Point Clé
        texte_pc = (
            contexte_regional +
            f"Pendant cette période, **{meilleur}** a la moyenne la plus élevée ({moyennes[meilleur]} µg/m³), "
            f"et **{plus_bas}** a la plus basse ({moyennes[plus_bas]} µg/m³)."
        )
        story[sections[0]] = {
            "titre": f"{meilleur} vs {plus_bas} : Point Clé",
            "texte": texte_pc,
            "chart": None
        }

        # Section 2 : Courbe Multi-lignes
        visu = generer_visu(df, "Comparer", audience, polluants_sel, None)
        texte_visu = (
            "Comparaison visuelle des évolutions quotidiennes de chaque polluant. "
            "Les lignes superposées permettent d’identifier rapidement si les pics sont synchronisés ou non."
        )
        story[sections[1]] = {
            "titre": "Tendances Comparées",
            "texte": texte_visu,
            "chart": visu
        }

        # Section 3 : Moyennes Chiffrées
        details = "\n".join([f"- {poll} : {moyennes[poll]} µg/m³" for poll in moyennes])
        story[sections[2]] = {
            "titre": "Moyennes par Polluant",
            "texte": f"Moyennes calculées sur la période :\n{details}",
            "chart": None
        }

        return story

    # --- Cas "Prévoir" ---
    if intention == "Prévoir":
        if not polluants_sel:
            story[sections[0]] = {
                "titre": "Aucune sélection de polluant",
                "texte": "Sélectionnez un polluant pour obtenir une prévision.",
                "chart": None
            }
            return story

        p = polluants_sel[0]
        base = df[df["polluant"] == p].sort_values("date").copy()
        if base.shape[0] < 2:
            story[sections[0]] = {
                "titre": f"Pas assez de données pour {p}",
                "texte": "Au moins deux points historiques sont nécessaires pour prévoir.",
                "chart": None
            }
            return story

        base["indice"] = np.arange(len(base))
        modele = LinearRegression().fit(base[["indice"]], base[["valeur"]])
        jours_futurs = {"Court Terme": 2, "Moyen Terme": 7, "Long Terme": 15}[forecast_sel]
        fut_idx = np.arange(len(base), len(base) + jours_futurs)
        preds = modele.predict(fut_idx.reshape(-1, 1)).flatten()
        fut_dates = pd.date_range(base["date"].max() + timedelta(days=1), periods=jours_futurs)

        hist = base[["date", "valeur"]].assign(type="Historique", polluant=p)
        fut = pd.DataFrame({
            "date": fut_dates,
            "valeur": preds,
            "type": "Prévision",
            "polluant": p
        })
        total = pd.concat([hist, fut], ignore_index=True)

        # Section 1 : Résumé chiffré + courbe
        moy_histo = round(base["valeur"].mean(), 1)
        moy_pred = round(np.mean(preds), 1)
        min_pred = round(preds.min(), 1)
        max_pred = round(preds.max(), 1)
        texte_intro = (
            contexte_regional +
            f"Historique du {debut.strftime('%d/%m/%Y')} au {fin.strftime('%d/%m/%Y')} pour **{p}** : moyenne = {moy_histo} µg/m³.\n"
            f"Prévision sur {jours_futurs} jours : moyenne prévue = {moy_pred} µg/m³, min = {min_pred} µg/m³, max = {max_pred} µg/m³."
        )
        visu = generer_visu(df, "Prévoir", audience, [p], forecast_sel)
        story[sections[0]] = {
            "titre": f"Prévisions pour {p}",
            "texte": texte_intro,
            "chart": visu
        }

        # Section 2 : Interprétation
        tendance_globale = "hausse" if moy_pred > moy_histo else "baisse" if moy_pred < moy_histo else "stabilité"
        bloc_narratif = (
            f"Tendance observée : {tendance_globale}. Si la tendance haussière se confirme, "
            "mettez en place des mesures temporaires (circulation alternée, messages de sensibilisation)."
        )
        story[sections[2]] = {
            "titre": "Interprétation",
            "texte": bloc_narratif,
            "chart": None
        }

        # Section 3 : Recommandations
        reco_texte = (
            "Recommandations :\n"
            "- Renforcer la communication publique.\n"
            "- Envisager des restrictions temporaires sur le trafic.\n"
            "- Surveiller les émissions industrielles."
        )
        story[sections[3]] = {
            "titre": "Recommandations",
            "texte": reco_texte,
            "chart": None
        }

        return story

    # --- Cas "Recommander" ---
    if intention == "Recommander":
        # DataFrame des actions
        actions = ["Sensibilisation", "Réglementation", "Transport Propre", "Optimisation Industrielle", "Vélo-Partage"]
        impacts = [20, 35, 25, 40, 15]
        precisions = [85, 87, 90, 82, 88]
        df_actions = pd.DataFrame({
            "Action": actions,
            "Impact Estimé (%)": impacts,
            "Précision DT (%)": precisions
        })

        # Section 1 : Contexte
        contexte_texte = (
            contexte_regional +
            "Pour améliorer la qualité de l'air, plusieurs actions peuvent être mises en place. "
            "Le tableau ci-dessous montre l’impact estimé et la précision du Digital Twin pour chaque action."
        )
        story[sections[0]] = {
            "titre": "Contexte des Actions",
            "texte": contexte_texte,
            "chart": None
        }

        # Section 2 : Tableau + Barres Colorées
        texte_tableau = (
            "Le tableau suivant présente chaque action avec son **impact estimé** (réduction potentielle) et la **précision du DT** "
            "(fiabilité de la prévision). Plus la précision est élevée, plus le résultat est fiable."
        )
        # Bar chart coloré selon la précision
        chart_actions = (
            alt.Chart(df_actions)
            .mark_bar(cornerRadius=4)
            .encode(
                y=alt.Y("Action:N", sort="-x", title="Action"),
                x=alt.X("Impact Estimé (%):Q", title="Impact Estimé (%)"),
                color=alt.Color("Précision DT (%):Q", title="Précision DT (%)",
                                scale=alt.Scale(scheme="viridis")),
                tooltip=["Action:N", "Impact Estimé (%):Q", "Précision DT (%):Q"]
            )
            .properties(width="container", height=300)
        )
        # Afficher aussi le tableau
        story[sections[1]] = {
            "titre": "Tableau des Actions",
            "texte": texte_tableau,
            "chart": (df_actions, chart_actions)  # on renvoie un tuple (DataFrame, Altair chart)
        }

        # Section 3 : Recommandation Prioritaire
        idx_max = df_actions["Précision DT (%)"].idxmax()
        action_prioritaire = df_actions.loc[idx_max, "Action"]
        precision_max = df_actions.loc[idx_max, "Précision DT (%)"]
        impact_corresp = df_actions.loc[idx_max, "Impact Estimé (%)"]
        reco_texte = (
            f"L’action la plus fiable est **{action_prioritaire}** (Précision DT = {precision_max} %), "
            f"avec un impact estimé de {impact_corresp} %. Nous recommandons de la prioriser pour maximiser le bénéfice."
        )
        story[sections[2]] = {
            "titre": "Recommandation Prioritaire",
            "texte": reco_texte,
            "chart": None
        }

        return story

    # --- Si intention invalide ---
    story["Erreur"] = {
        "titre": "Intention inconnue",
        "texte": "Sélectionnez une intention valide (Décrire, Comparer, Prévoir ou Recommander).",
        "chart": None
    }
    return story


# --- PAGE PRINCIPALE ---
st.title("🌿 Qualité de l'Air Régionale")
st.markdown("Interagissez avec le Jumeau Numérique pour bénéficier d’analyses sur mesure et d’interprétations limpides, adaptées à votre profil.")

# --- SECTION 1 : Carte Interactive ---
st.markdown("### 🗺️ Carte Interactive de Concentrations des polluants Régionales")
col_map1, col_map2 = st.columns([3, 1])
with col_map2:
    region_map_sel = st.selectbox("Région (Carte)", list(REGIONS.keys()), index=0)
    filtre_poll = st.multiselect("Filtrer par polluant", POLLUANTS, default=POLLUANTS)
    filtre_date_type = st.selectbox("Granularité", ["Aucune", "Journalier", "Saisonnier"], index=0)

lat0_map, lon0_map = REGIONS[region_map_sel]
df_map = pd.DataFrame([
    {
        "lat": float(lat0_map + np.random.randn() * 0.3),
        "lon": float(lon0_map + np.random.randn() * 0.3),
        "valeur": float(np.random.uniform(20, 80)),
        "polluant": str(np.random.choice(filtre_poll)) if filtre_poll else None,
        "jour": (datetime(2025, 1, 1) + timedelta(days=int(np.random.uniform(0, 30)))).strftime("%Y-%m-%d")
    }
    for _ in range(300)
])
if filtre_poll:
    df_map = df_map[df_map["polluant"].isin(filtre_poll)]
else:
    df_map = pd.DataFrame()

if filtre_date_type == "Journalier" and not df_map.empty:
    df_map = df_map[df_map["jour"] == "2025-01-25"]
elif filtre_date_type == "Saisonnier" and not df_map.empty:
    debut_trim = datetime(2025, 1, 1).date()
    fin_trim = datetime(2025, 3, 31).date()
    df_map = df_map[
        (pd.to_datetime(df_map["jour"]) >= pd.to_datetime(debut_trim)) &
        (pd.to_datetime(df_map["jour"]) <= pd.to_datetime(fin_trim))
    ]

if not df_map.empty:
    palette = {"O3": [220, 53, 69], "PM10": [40, 167, 69], "NH3": [23, 162, 184], "PM2.5": [255, 193, 7], "CH4": [108, 117, 125]}
    df_map["couleur"] = df_map["polluant"].map(lambda x: palette.get(x, [100, 100, 100]) if x else [100, 100, 100])
    df_map["couleur_rgba"] = df_map["couleur"].apply(lambda c: [c[0], c[1], c[2], 150])

    layer = pdk.Layer(
        "ScatterplotLayer",
        data=df_map,
        get_position="[lon, lat]",
        get_radius=5000,
        get_fill_color="couleur_rgba",
        pickable=True,
        auto_highlight=True
    )
    tooltip = {"html": "<b>Polluant :</b> {polluant}<br/><b>Valeur :</b> {valeur} µg/m³", "style": {"backgroundColor": "white", "color": "black"}}
    initial_view = {"latitude": lat0_map, "longitude": lon0_map, "zoom": 6}
    with col_map1:
        st.pydeck_chart(pdk.Deck(layers=[layer], initial_view_state=initial_view, tooltip=tooltip), use_container_width=True)

    legende_html = '<div class="legend">'
    for p in filtre_poll:
        color = palette.get(p, [100, 100, 100])
        legende_html += (
            f'<div class="legend-item">'
            f'<div class="legend-color" style="background-color:rgba({color[0]},{color[1]},{color[2]},0.6)"></div>{p}'
            f'</div>'
        )
    legende_html += '</div>'
    st.markdown(legende_html, unsafe_allow_html=True)
else:
    with col_map1:
        st.info("Aucun point à afficher pour la configuration sélectionnée.")
st.markdown("---")

# --- SECTION 2 : Personnalisez votre requête ---
with st.form(key="story_form"):
    st.markdown("""
### 🔧 Paramètres de Requête
- **Région** : zone géographique.
- **Polluant(s) ciblé(s)** : choisissez un ou plusieurs polluants.
- **Profil** : les interprétations générées seront adaptées aux publics grand public ou aux experts du domaine.
- **Objectif** :
  - **Décrire** : aperçu des niveaux actuels.
  - **Comparer** : tendances relatives.
  - **Prévoir** : estimer l’évolution future.
  - **Recommander** : recommandations d’actions.
""", unsafe_allow_html=True)

    col1, col2, col3, col4 = st.columns(4)
    with col1:
        region_sel = st.selectbox("Région (Story)", list(REGIONS.keys()), index=0)
    with col2:
        polluants_sel = st.multiselect("Polluant(s) ciblé(s)", POLLUANTS, default=["O3", "PM10"])
    with col3:
        audience_sel = st.selectbox("Profil", PROFILS)
    with col4:
        intent_sel = st.selectbox("Objectif", INTENTS)

    st.markdown("---")
    col5, col6, col7, col8 = st.columns(4)
    with col5:
        date_debut = st.date_input("Du", datetime(2025, 1, 1))
    with col6:
        date_fin = st.date_input("Au", datetime(2025, 1, 31))
    with col7:
        forecast_sel = st.selectbox("Prévision (si applicable)", FORECASTS, disabled=(intent_sel != "Prévoir"))
    with col8:
        sources_sel = st.multiselect("Sources (Expert)", SOURCES, disabled=(audience_sel != "Expert"))

    submit = st.form_submit_button("🚀 Générer la Data Story")

if submit:
    df_donnees = charger_donnees(polluants_sel, date_debut, date_fin)
    moyennes = {
        p: round(val, 1)
        for p, val in df_donnees.groupby("polluant")["valeur"].mean().items()
    }
    seuil_critique = 60.0
    poll_critiques = [p for p, m in moyennes.items() if m >= seuil_critique]

    st.markdown("---")
    st.subheader("🔔 Alerte & Niveau de Pollution")
    if poll_critiques:
        lignes = [f"• **{p}** : {moyennes[p]} µg/m³ (au-dessus du seuil critique)" for p in poll_critiques]
        msg_alerte = (
            "🚨 **Situation d’alerte pour :**\n\n"
            + "\n".join(lignes)
            + "\n\n> Réduisez les activités extérieures et suivez les consignes."
        )
        st.warning(msg_alerte)
    else:
        st.success("✅ Pollution sous contrôle pour tous les polluants sélectionnés.")

    # Titre dynamique
    titre_main = f"📖 {intent_sel} – {', '.join(polluants_sel)} en {region_sel}"
    st.header(titre_main)

    # Génération de la data story
    story = generer_story_vraie(
        df_donnees,
        intent_sel,
        MAPPING_STRUCTURE[intent_sel],
        audience_sel,
        region_sel,
        date_debut,
        date_fin,
        polluants_sel,
        forecast_sel
    )

    # Affichage des sections
    for cle, contenu in story.items():
        titre = contenu.get("titre", cle)
        texte = contenu.get("texte", "")
        chart = contenu.get("chart", None)

        st.subheader(titre)
        st.markdown(texte)

        # Afficher le contenu visuel
        if isinstance(chart, pd.DataFrame):
            st.table(chart)
        elif isinstance(chart, tuple) and isinstance(chart[0], pd.DataFrame) and isinstance(chart[1], alt.Chart):
            df_actions, bar_chart = chart
            st.table(df_actions)
            st.altair_chart(bar_chart, use_container_width=True)
        elif isinstance(chart, alt.VConcatChart) or isinstance(chart, alt.Chart):
            st.altair_chart(chart.interactive(), use_container_width=True)
        else:
            pass

    # Suggestion des visuels
    types_viz = VIZ_RECOMMANDEES[intent_sel][audience_sel]
    st.info(f"🔍 Visuels recommandés : {', '.join(types_viz)}")


In [ ]:
!nohup streamlit run dashboard_dt.py --server.port 8502 &

In [ ]:
from pyngrok import ngrok
ngrok.kill()  # Fermer d’éventuels tunnels précédents

# Authentification (tu l’as déjà bien mise)
ngrok.set_auth_token("ngroktoken")

# Connexion normale, sans sous-domaine personnalisé
public_url = ngrok.connect(8502)

print("Votre application Streamlit est accessible à l'adresse :", public_url)